In [1]:
from PIL import Image
import pandas as pd
from scipy import signal
from pathlib import Path
from tqdm import tqdm  
from os import path
from myfunctions import event_separation , image_crop_save_gauss , image_crop_save , image_crop_negative, poi, delete_old_extracted_events
import copy

from database.extract_yaml import get_dict, save_dict


## Input data from a folder ##

In [2]:
BASE_DIR = r'//lebnas1.epfl.ch/microsc125/deep_events'
SAVING_SCHEME = "ws_0.1"

In [3]:
files_dir = os.path.join(BASE_DIR, 'original_data')
images_dir = '220922_mtstaygold_U2OS_iSIM_fl'
joined_path = os.path.join(files_dir, images_dir)
size=(2048,2048)
img,input_name,output_name,datacsv={},{},{},{}

# Only look at ome.tif and csv files
path = Path(joined_path)
files = list(path.glob("*.ome.tif"))
files.extend(list(path.glob("*.csv")))


for input_file in files:
    input_file = str(input_file.name)
    joined_file_path = os.path.join(files_dir, images_dir, input_file)
    if '.tif' in input_file:
        date, cell_type, dye, bf_fl, index  = input_file.split('_')
        number, ome, tiff= index.split('.')
        img[int(number)-1] = Image.open(joined_file_path)
        input_name[int(number)-1]= f'points_{date}_{cell_type}_{bf_fl}_{number}'
        output_name[int(number)-1]= f'image_{date}_{cell_type}_{bf_fl}_{number}'
        print('Loaded image:'+input_file)
    elif '.yaml' in input_file:
        pass
    else:
        labels, index, bf_fl  = input_file.split('_')
        datacsv[int(index)-1] = pd.read_csv(joined_file_path)
        print('Loaded csv:'+ input_file)
max_number=int(number)        


Loaded image:220922_mtstaygold_U2OS_fl_1.ome.tif
Loaded image:220922_mtstaygold_U2OS_fl_2.ome.tif
Loaded image:220922_mtstaygold_U2OS_fl_3.ome.tif
Loaded image:220922_mtstaygold_U2OS_fl_4.ome.tif
Loaded image:220922_mtstaygold_U2OS_fl_5.ome.tif
Loaded csv:mtStayGold_1_points.csv
Loaded csv:mtStayGold_2_points.csv
Loaded csv:mtStayGold_3_points.csv
Loaded csv:mtStayGold_4_points.csv
Loaded csv:mtStayGold_5_points.csv


## Get all Gaussian Points of Interest ##

In [ ]:
print(max_number)
for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    for i in range(5,6,1):
        sigma=(i,i)
        s=sigma[0]
        in_name=os.path.join(BASE_DIR, 'original_data', images_dir, f'{input_name[x]}_sigma{s}.tiff')
        framenum=im.n_frames
        poi(csv,in_name,sigma,size,framenum)

## Cut Images and Gaussian POIs ##

In [5]:
base_dir = BASE_DIR
training_dir = os.path.join(base_dir, 'training_data')
folder_name = images_dir+'_pos'
filepath = os.path.join(training_dir , folder_name)


tif_files = list(item for item in files if ".tif" in item.name)

folder_dict = get_dict(images_dir)
delete_old_extracted_events(folder_dict, os.path.join(training_dir, images_dir))
event_dict = copy.deepcopy(folder_dict)
event_dict['type'] = "event"
del event_dict['extracted_events']

for x in range(max_number):
    event_dict['original_file'] = os.path.basename(tif_files[x])
    event_dict['event_content'] = 'division'
    csv=datacsv[x]
    im=img[x]
    
    out_name=f'{output_name[x]}'           
    list1=event_separation(csv)
    l=len(list1)
    print(f"number of events: {l}")
    event_path = image_crop_save(l,list1, csv, im, out_name,filepath, SAVING_SCHEME=SAVING_SCHEME, 
                        folder_dict=folder_dict, event_dict=event_dict)

    
    for i in range(3,4,1):
        sigma=(i,i)
        s=sigma[0]
        in_name=f'{input_name[x]}_sigma{s}.tiff'
        out_name_g=f'{output_name[x]}_sigma{s}'
    

        # gauss=base_dir+f'\{in_name}'
        # gauss_image=Image.open(gauss)
        # image_crop_save_gauss(l,list1, csv, gauss_image, out_name_g,filepath)

removing //lebnas1.epfl.ch/microsc125/deep_events\training_data\220922_6397664d0c3422bd59edc935.
removing //lebnas1.epfl.ch/microsc125/deep_events\training_data\220922_639766920c3422bd59edc936.
removing //lebnas1.epfl.ch/microsc125/deep_events\training_data\220922_639766920c3422bd59edc937.
removing //lebnas1.epfl.ch/microsc125/deep_events\training_data\220922_639766920c3422bd59edc938.
removing //lebnas1.epfl.ch/microsc125/deep_events\training_data\220922_639766920c3422bd59edc939.
removing //lebnas1.epfl.ch/microsc125/deep_events\training_data\220922_639766920c3422bd59edc93a.
removing //lebnas1.epfl.ch/microsc125/deep_events\training_data\220922_639766920c3422bd59edc93b.
removing //lebnas1.epfl.ch/microsc125/deep_events\training_data\220922_639766930c3422bd59edc93c.
removing //lebnas1.epfl.ch/microsc125/deep_events\training_data\220922_639766930c3422bd59edc93d.
removing //lebnas1.epfl.ch/microsc125/deep_events\training_data\220922_639766940c3422bd59edc93e.
removing //lebnas1.epfl.ch/mic

## Cut Negative Frames ##

In [6]:
folder_name=images_dir+'_neg'
filepath= os.path.join(training_dir , folder_name)
from myfunctions import image_crop_negative

folder_dict = get_dict(images_dir)
event_dict = copy.deepcopy(folder_dict)
event_dict['type'] = "event"
del event_dict['extracted_events']


for x in range(max_number):
    event_dict['event_content'] = 'negative'
    event_dict['original_file'] = os.path.basename(tif_files[x])
    csv=datacsv[x]
    im=img[x]
    out_name=f'{output_name[x]}'

    list1=event_separation(csv)
    l=len(list1)
    image_crop_negative(l,list1, csv, im, out_name,filepath)

[0, 1, 2, 3, 4, 5, 6, 7]
   axis-0       axis-1       axis-2
0     8.0  1117.563098  1416.049878
1     7.0  1117.563098  1416.049878
2     6.0  1117.058833  1415.713701
3     5.0  1117.395010  1416.217966
4     4.0  1117.563098  1416.386055
5     3.0  1117.226921  1415.209435
6     2.0  1117.563098  1415.881789
[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
    axis-0       axis-1       axis-2
8     71.0  1114.580301  1466.444429
9     72.0  1114.580301  1466.444429
10    73.0  1114.143631  1466.444429
11    74.0  1113.852518  1465.862203
12    75.0  1113.197514  1466.080538
13    76.0  1112.178618  1467.390546
14    69.0  1105.854567  1466.355333
15    68.0  1105.854567  1468.235597
16    67.0  1106.296982  1467.793181
17    66.0  1106.518190  1468.346200
18    65.0  1108.398454  1469.341634
[20, 21, 22, 23, 24, 25]
    axis-0       axis-1       axis-2
20     9.0  1060.628968  1514.424658
21    10.0  1061.071384  1511.770167
22    11.0  1062.841044  1508.009639
23    12.0  1063.172855